In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pymongo
import re
import pandas as pd
import requests

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

def soup_url (url):
    browser = init_browser()
    # Visit url
    browser.visit(url)
    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup_url = bs(html, "html.parser")
    
     # Close the browser after scraping
    browser.quit()
    
    return soup_url

def soup_class (soup, class_name):
    
    # Soup data list
    soup_class = soup.find(class_= class_name)
    
    return soup_class 

def soup_class_list (soup, class_name):
   
    # Soup data list
    soup_class_list = soup.find_all(class_= class_name)
    
    return soup_class_list

def text_list (soup_class_list):
    text_list = []
    [text_list.append(record.text.replace("\n", "")) for record in soup_class_list]
    return text_list

# WEB Scrapping Latest Oil News

In [3]:
# Visit oilprice.com
base_url = "https://oilprice.com"
url_news = "https://oilprice.com/Latest-Energy-News/World-News/"
soup = soup_url(url_news)

In [4]:
page_title = soup.title.text
page_title

'Oil News Today  | OilPrice.com'

In [5]:
image_holder = soup.find_all('img', src=True)
image_holder

[<img alt="Crude oil prices today - Oilprice.com" class="desktop_logo" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/oilprice-logo.png?v=3" title="Oil prices - Oilprice.com"/>,
 <img alt="Crude oil prices today - Oilprice.com" class="mobile_logo" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/mobile_logo.png?v=3" title="Oil prices - Oilprice.com"/>,
 <img alt="OilPrice GEA" id="main_nav_button" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/nav_button_green.png" style="height: 25px;" title="Global Energy Alert"/>,
 <img alt="Link" height="21" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/link.png" width="21"/>,
 <img alt="Link" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/link.png"/>,
 <img alt="Link" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/link.png"/>,
 <img alt="Link" height="21" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/link.png" width="21"/>,
 <img alt="Link" height="21" src="

In [6]:
# Scrapping News Data
articles = soup.find_all(class_= "categoryArticle")
title_list = []
date_list = []
excerpt_list = []
author_list = []
link_list = []
for article in articles:
    image_holder = article.find(class_= "categoryArticle__imageHolder")
    img =image_holder.findAll('img')
    link = img[0].get('data-src')
    link_list.append(link)
    title = article.find_all(class_= "categoryArticle__title")
    full_date = article.find_all(class_= "categoryArticle__meta")
    date = full_date[0].text.strip().split("|")[0]
    author = full_date[0].text.strip().split("|")[1]
    excerpt = article.find_all(class_= "categoryArticle__excerpt")
    title_list.append(title[0].text.strip())
    date_list.append(date)
    author_list.append(author)
    excerpt_list.append(excerpt[0].text.strip())
link_list

['https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-09_amtgbxq0jv.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-09_pmmcvsuii9.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-09_x21hgxy496.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-09_25zs7tooxw.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-09_5htbeuygso.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-08_ygt0erondp.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-08_xbgja9u6ri.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-08_rbg9uv6tkp.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-08_ddu9cwlbvo.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-08_tzsfkjgoqe.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-08_gteqvhpzrb.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-07_fbciprydsj.jpg',
 'https://d32r1sh890xpii.clo

In [7]:
news = []
for i in range(len(title_list)):
    row = {}
    row['Date'] = date_list[i]
    row["News_Title"] = title_list[i]
    row["News_Paragraph"] = excerpt_list[i]
    row["Author"] = author_list[i]
    row["Image_URL"] = link_list[i]
    news.append(row)
news[:5]

[{'Date': 'Dec 09, 2020 at 14:05 ',
  'News_Title': 'Big Oil Boosts Earnings From Tax-Haven Affiliates',
  'News_Paragraph': 'Some of the world’s biggest oil corporations have boosted their earnings in recent years by paying very little or no taxes at all on profits of their affiliates in offshore…',
  'Author': ' Charles Kennedy',
  'Image_URL': 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-09_amtgbxq0jv.jpg'},
 {'Date': 'Dec 09, 2020 at 13:19 ',
  'News_Title': 'Kurdistan To Hand Over Oil Revenue To Iraq In Exchange For Funds',
  'News_Paragraph': 'The Kurdistan Regional Government (KRG) will comply with the recent Iraqi federal fiscal deficit bill under which the semi-autonomous region in Iraq is set to hand over an "unspecified amount…',
  'Author': ' Tsvetana Paraskova',
  'Image_URL': 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-12-09_pmmcvsuii9.jpg'},
 {'Date': 'Dec 09, 2020 at 12:06 ',
  'News_Title': 'Maduro Accuses CIA Of Bribing Oil Workers',
  'New

# WEB Scrapping Latest Oil Prices

In [8]:
# Visit oilprice.com
url_prices = "https://oilprice.com/oil-price-charts/#prices"
soup_prices = soup_url(url_prices)

In [9]:
page_title = soup_prices.title.text
page_title

'Oil Price Charts | Oilprice.com'

In [10]:
price_holders = soup_prices.find_all(class_= "stripe show_graph update_on_load")
price_holders[:2]

[<tr class="stripe show_graph update_on_load" data-id="45" data-spreadsheet="Crude Oil WTI">
 <td class="flag_holder"><div class="flag_usa"></div></td><td>WTI Crude<i aria-hidden="true" class="fa fa-plus user_add_blend hidden" style="float:right;" title="Follow blend"></i><i aria-hidden="true" class="fa fa-times user_remove_blend hidden" style="float:right;" title="Unfollow blend"></i></td>
 <td class="last_price" data-price="45.64">45.64</td><td class="change_up flat_change_cell" dir="ltr">+0.04</td><td class="change_up_percent percent_change_cell" dir="ltr">+0.09%<span class="blend_update_text">(10 Minutes Delay)</span></td><td class="last_updated" data-stamp="1607550000">(10 Minutes Delay)</td></tr>,
 <tr class="stripe show_graph update_on_load" data-id="46" data-spreadsheet="Crude Oil Brent">
 <td class="flag_holder"><div class="flag_uk"></div></td><td>Brent Crude<i aria-hidden="true" class="fa fa-plus user_add_blend hidden" style="float:right;" title="Follow blend"></i><i aria-hid

In [11]:
r = requests.get(url_prices)
price_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
price_df = price_list[0]
price_df

,0,1,2,3,4,5
0,NaN,WTI Crude,45.64,+0.04,+0.09%(10 Minutes Delay),(10 Minutes Delay)
1,NaN,Brent Crude,48.98,+0.14,+0.29%(11 Minutes Delay),(11 Minutes Delay)
2,NaN,Mars US,46.40,-0.01,-0.02%(21 \tHours Delay),(21 \tHours Delay)
3,NaN,Opec Basket,47.79,-0.56,-1.16%(1 day Delay),(1 day Delay)
4,NaN,Canadian Crude Index,32.22,-0.65,-1.98%(2 \tweeks Delay),(2 \tweeks Delay)
5,NaN,DME Oman,48.90,+0.24,+0.49%(1 day Delay),(1 day Delay)
6,NaN,Urals,39.60,+0.00,+0.00%(3 \tweeks Delay),(3 \tweeks Delay)
7,NaN,Mexican Basket,44.17,+0.01,+0.02%(1 day Delay),(1 day Delay)
8,NaN,Indian Basket,48.50,-0.02,-0.04%(1 day Delay),(1 day Delay)
9,NaN,Western Canadian Select,33.45,-0.11,-0.33%(14 \tHours Delay),(14 \tHours Delay)


In [12]:
prices = []
for price_row in price_df.iterrows():
    row = {}
    row['Oil_Symbol'] = price_row[1][1]
    row['Oil_Price'] = price_row[1][2]
    row['Oil_Change_Value'] = price_row[1][3]
    row['Oil_Change_Interest'] = price_row[1][4].split("(")[0]
    row['Oil_Price_Delay'] = price_row[1][5]
    prices.append(row)
prices[:10]

[{'Oil_Symbol': 'WTI Crude',
  'Oil_Price': '45.64',
  'Oil_Change_Value': '+0.04',
  'Oil_Change_Interest': '+0.09%',
  'Oil_Price_Delay': '(10 Minutes Delay)'},
 {'Oil_Symbol': 'Brent Crude',
  'Oil_Price': '48.98',
  'Oil_Change_Value': '+0.14',
  'Oil_Change_Interest': '+0.29%',
  'Oil_Price_Delay': '(11 Minutes Delay)'},
 {'Oil_Symbol': 'Mars US',
  'Oil_Price': '46.40',
  'Oil_Change_Value': '-0.01',
  'Oil_Change_Interest': '-0.02%',
  'Oil_Price_Delay': '(21 \tHours Delay)'},
 {'Oil_Symbol': 'Opec Basket',
  'Oil_Price': '47.79',
  'Oil_Change_Value': '-0.56',
  'Oil_Change_Interest': '-1.16%',
  'Oil_Price_Delay': '(1 day Delay)'},
 {'Oil_Symbol': 'Canadian Crude Index',
  'Oil_Price': '32.22',
  'Oil_Change_Value': '-0.65',
  'Oil_Change_Interest': '-1.98%',
  'Oil_Price_Delay': '(2 \tweeks Delay)'},
 {'Oil_Symbol': 'DME Oman',
  'Oil_Price': '48.90',
  'Oil_Change_Value': '+0.24',
  'Oil_Change_Interest': '+0.49%',
  'Oil_Price_Delay': '(1 day Delay)'},
 {'Oil_Symbol': 'Ural

# WEB Scrapping Energy Companies News

In [13]:
# Visit Baker Hughes
base_url = "https://www.bakerhughes.com"
url_news = "https://www.bakerhughes.com/company/energy-forward"
soup = soup_url(url_news)

In [14]:
page_title = soup.title.text
page_title

'Energy Forward Stories | Baker Hughes'

In [15]:
articles = soup.find_all(class_= "article-tile-ef__top-container")
articles_sum = soup.find_all(class_= "article-tile-ef__metadata-summary")

In [16]:
title_list = []
link_list = []
for article in articles:
    img = article.findAll('img')
    link = img[0].get('src')
    link_list.append(link)
    title = article.find_all(class_= "article-tile-ef__title")
    title_list.append(title[0].text.strip())
title_list

['We’ve only scratched the surface using AI and dron',
 'Applying AI in a pandemic',
 '8 things you should know about hydrogen energy',
 'Straight talk on decarbonization']

In [17]:
text_list = []
for article in articles_sum:
    full_text = article.find_all(class_= "article-tile-ef__summary")
    text_list.append(full_text[0].text.strip())
text_list

['Vice President Safety and Environment Unconventionals at Shell, Kevin McMahon talks about deploying drones for methane detection in the Permian Basin.',
 'An alliance of technologists applies artificial intelligence (AI) to the pivotal task, achieving benefits on every front.',
 'CEO Richard Dyson, and Mary Snowdon, Principal Consultant at io consulting list what to consider before joining the hydrogen revolution.',
 'Rod Christie, Executive Vice President of Turbomachinery & Process Solutions wants you to know the oil & gas industry is serious about climate change.']

In [18]:
baker = [{'News_Title': 'Baker Hughes Company (BKR), an energy technology company', 
         'News_Paragraph': 'Our portfolio of technology, equipment and services solves customer challenges in energy and industrial applications.',
         'Image_URL': 'https://www.bakerhughes.com/sites/bakerhughes/files/styles/max_2600x2600/public/2019-09/09242019_aboutus_i4%402x.jpg'
         }]
for i in range(len(title_list)):
    row = {}
    row["News_Title"] = title_list[i]
    row["News_Paragraph"] = text_list[i]
    row["Image_URL"] = link_list[i]
    baker.append(row)
baker

[{'News_Title': 'Baker Hughes Company (BKR), an energy technology company',
  'News_Paragraph': 'Our portfolio of technology, equipment and services solves customer challenges in energy and industrial applications.',
  'Image_URL': 'https://www.bakerhughes.com/sites/bakerhughes/files/styles/max_2600x2600/public/2019-09/09242019_aboutus_i4%402x.jpg'},
 {'News_Title': 'We’ve only scratched the surface using AI and dron',
  'News_Paragraph': 'Vice President Safety and Environment Unconventionals at Shell, Kevin McMahon talks about deploying drones for methane detection in the Permian Basin.',
  'Image_URL': 'https://www.bakerhughes.com/sites/bakerhughes/files/styles/product_services_thumbnail/public/2020-09/Avitas%20Drone%20Aug%202020%2023.JPG?itok=bvQ5skEv'},
 {'News_Title': 'Applying AI in a pandemic',
  'News_Paragraph': 'An alliance of technologists applies artificial intelligence (AI) to the pivotal task, achieving benefits on every front.',
  'Image_URL': 'https://www.bakerhughes.co

In [19]:
# Visit Chevron
base_url = "https://www.chevron.com"
url_news = "https://www.chevron.com/projects"
soup = soup_url(url_news)

In [20]:
page_title = soup.title.text
page_title

'Chevron International Project Portfolio  — Chevron.com'

In [41]:
articles = soup.find_all(class_= "c12 container-fluid  section-bg-color bg-white")

[]

In [45]:
title_list = ['Project Gorgon', 'Project the Permian Basin', 'Project Tengiz Expansion', 'Project Jack/St.Malo']
text_list = ['One of the largest global natural gas projects is primed  to power the Asia-Pacific region.',
             'An engine of America’s energy resurgence is delivering transformational growth for us.',
             'For more than 20 years, we’ve been operating one of the  world’s deepest supergiant oil fields and supporting local communities in  Kazakhstan.',
             'In the U.S. Gulf of Mexico, we’re taking deepwater  technology to new heights—nearly 30,000 feet beneath the ocean’s surface.'
            ]
link_list = ['https://www.chevron.com/projects/-/media/chevron/projects/images/gorgon-tile.jpg', 
             'https://www.chevron.com/projects/-/media/chevron/projects/images/permian-project-tile.jpg',
             'https://www.chevron.com/projects/-/media/chevron/projects/images/tengizchevroil-tile.jpg',
             'https://www.chevron.com/projects/-/media/chevron/projects/images/jack-st-malo-project-tile.jpg',]

In [63]:
chevron = [{'News_Title': 'Chevron Corporation (CVX)', 
            'News_Paragraph': 'From upstream exploration and production to midstream transportation, power and trading to downstream manufacturing and retail, each of our businesses work in concert across virtually every facet of the energy industry.',
            'Image_URL': 'https://www.chevron.com/-/media/chevron/investors/images/ir-hero-platong-platform.jpg'}]
for i in range(len(title_list)):
    row = {}
    row["News_Title"] = title_list[i]
    row["News_Paragraph"] = text_list[i]
    row["Image_URL"] = link_list[i]
    chevron.append(row)
chevron

[{'News_Title': 'Chevron Corporation (CVX)',
  'News_Paragraph': 'From upstream exploration and production to midstream transportation, power and trading to downstream manufacturing and retail, each of our businesses work in concert across virtually every facet of the energy industry.',
  'Image_URL': 'https://www.chevron.com/-/media/chevron/investors/images/ir-hero-platong-platform.jpg'},
 {'News_Title': 'Project Gorgon',
  'News_Paragraph': 'One of the largest global natural gas projects is primed  to power the Asia-Pacific region.',
  'Image_URL': 'https://www.chevron.com/projects/-/media/chevron/projects/images/gorgon-tile.jpg'},
 {'News_Title': 'Project the Permian Basin',
  'News_Paragraph': 'An engine of America’s energy resurgence is delivering transformational growth for us.',
  'Image_URL': 'https://www.chevron.com/projects/-/media/chevron/projects/images/permian-project-tile.jpg'},
 {'News_Title': 'Project Tengiz Expansion',
  'News_Paragraph': 'For more than 20 years, we’ve

In [50]:
# Visit Exxon
base_url = "https://corporate.exxonmobil.com"
url_news = "https://corporate.exxonmobil.com/Company/Who-we-are"
soup = soup_url(url_news)

In [51]:
page_title = soup.title.text
page_title

'Who we are | ExxonMobil'

In [53]:
exxon = [{'News_Title': 'Exxon Mobil Corporation (XOM)',
          'News_Paragraph': 'One of the world’s largest publicly traded energy providers and chemical manufacturers, develops and applies next-generation technologies to help safely and responsibly meet the world’s growing needs for energy and high-quality chemical products.',
          'Image_URL': 'https://corporate.exxonmobil.com/Company/Who-we-are/-/media/Global/Images/Article-top-banners/Natural-gas/Loading-dock-at-PNG-LNG_banner_90512.jpg'}]

In [54]:
url_news = 'https://finance.yahoo.com/quote/EOG/news/'
eog = [{'News_Title': 'EOG Resources, Inc. (EOG)',
        'News_Paragraph':'One of the best "shale" producers is a solid bet for a potential recovery in 2021. The company s principal producing areas are located in New Mexico, North Dakota, Texas, and Wyoming in the United States; and the Republic of Trinidad and Tobago, the People s Republic of China, and Canada',
        'Image_URL': 'https://s7d2.scene7.com/is/image/DNB/Industry-OilandGas-hero'}]

In [55]:
conoco = [{'News_Title':  'ConocoPhillips (COP)',
           'News_Paragraph': 'ConocoPhillips is a multinational corporation engaged in hydrocarbon exploration. It is based in the Energy Corridor district of Houston, Texas. ',
           'Image_URL': 'https://static.conocophillips.com/files/pages/ultrawide-med-19sr-report-cover-aplng.jpg'}]

In [56]:
valero = [{'News_Title': 'Valero Energy Corporation (VLO)',
           'News_Paragraph': 'Founded in 1980 and named for the mission San Antonio de Valero – the original name of the Alamo – Valero Energy Corporation has continued to grow and evolve to become the largest global independent petroleum refiner and the largest renewable fuels producer in North America.',
           'Image_URL': 'https://www.valero.com/sites/default/files/2020-04/About_SuppliersMineralInterestInvestRecov_700X550.jpg'}]

# Store Data in Database

In [21]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [73]:
# BAKER HUGHES
# Define database and collection
db = client.oil_db
collection = db.baker_intro_new
# Upload collection
for record in baker:
    collection.insert_one(record)
# Display the MongoDB records created above
baker_news_db = db.baker_intro_new.find()
for record in baker_news_db:
    print(record)

{'_id': ObjectId('5fb484ee094f647d970de9b7'), 'News_Title': 'Baker Hughes Company (BKR), an energy technology company', 'News_Paragraph': 'Our portfolio of technology, equipment and services solves customer challenges in energy and industrial applications.', 'Image_URL': 'https://www.bakerhughes.com/sites/bakerhughes/files/styles/max_2600x2600/public/2019-09/09242019_aboutus_i4%402x.jpg'}
{'_id': ObjectId('5fb484ee094f647d970de9b8'), 'News_Title': 'We’ve only scratched the surface using AI and dron', 'News_Paragraph': 'Vice President Safety and Environment Unconventionals at Shell, Kevin McMahon talks about deploying drones for methane detection in the Permian Basin.', 'Image_URL': 'https://www.bakerhughes.com/sites/bakerhughes/files/styles/product_services_thumbnail/public/2020-09/Avitas%20Drone%20Aug%202020%2023.JPG?itok=bvQ5skEv'}
{'_id': ObjectId('5fb484ee094f647d970de9b9'), 'News_Title': 'Applying AI in a pandemic', 'News_Paragraph': 'An alliance of technologists applies artificia

In [74]:
# CHEVRON
# Define database and collection
db = client.oil_db
collection = db.chevron_intro_new
# Upload collection
for record in chevron:
    collection.insert_one(record)
# Display the MongoDB records created above
baker_news_db = db.chevron_intro_new.find()
for record in baker_news_db:
    print(record)

{'_id': ObjectId('5fb4850d094f647d970de9bc'), 'News_Title': 'Chevron Corporation (CVX)', 'News_Paragraph': 'From upstream exploration and production to midstream transportation, power and trading to downstream manufacturing and retail, each of our businesses work in concert across virtually every facet of the energy industry.', 'Image_URL': 'https://www.chevron.com/-/media/chevron/investors/images/ir-hero-platong-platform.jpg'}
{'_id': ObjectId('5fb4850d094f647d970de9bd'), 'News_Title': 'Project Gorgon', 'News_Paragraph': 'One of the largest global natural gas projects is primed  to power the Asia-Pacific region.', 'Image_URL': 'https://www.chevron.com/projects/-/media/chevron/projects/images/gorgon-tile.jpg'}
{'_id': ObjectId('5fb4850d094f647d970de9be'), 'News_Title': 'Project the Permian Basin', 'News_Paragraph': 'An engine of America’s energy resurgence is delivering transformational growth for us.', 'Image_URL': 'https://www.chevron.com/projects/-/media/chevron/projects/images/perm

In [59]:
# CONOCO PHILIPS
# Define database and collection
db = client.oil_db
collection = db.conoco_intro
# Upload collection
for record in conoco:
    collection.insert_one(record)
# Display the MongoDB records created above
baker_news_db = db.conoco_intro.find()
for record in baker_news_db:
    print(record)

{'_id': ObjectId('5fb41520094f647d970de9b3'), 'News_Title': 'ConocoPhillips (COP)', 'News_Paragraph': 'ConocoPhillips is a multinational corporation engaged in hydrocarbon exploration. It is based in the Energy Corridor district of Houston, Texas. ', 'Image_URL': 'https://static.conocophillips.com/files/pages/ultrawide-med-19sr-report-cover-aplng.jpg'}


In [60]:
# EXXON MOBILE
# Define database and collection
db = client.oil_db
collection = db.exxon_intro
# Upload collection
for record in exxon:
    collection.insert_one(record)
# Display the MongoDB records created above
baker_news_db = db.exxon_intro.find()
for record in baker_news_db:
    print(record)

{'_id': ObjectId('5fb41556094f647d970de9b4'), 'News_Title': 'Exxon Mobil Corporation (XOM)', 'News_Paragraph': 'One of the world’s largest publicly traded energy providers and chemical manufacturers, develops and applies next-generation technologies to help safely and responsibly meet the world’s growing needs for energy and high-quality chemical products.', 'Image_URL': 'https://corporate.exxonmobil.com/Company/Who-we-are/-/media/Global/Images/Article-top-banners/Natural-gas/Loading-dock-at-PNG-LNG_banner_90512.jpg'}


In [61]:
# VALERO ENERGY
# Define database and collection
db = client.oil_db
collection = db.valero_intro
# Upload collection
for record in valero:
    collection.insert_one(record)
# Display the MongoDB records created above
baker_news_db = db.valero_intro.find()
for record in baker_news_db:
    print(record)

{'_id': ObjectId('5fb4157b094f647d970de9b5'), 'News_Title': 'Valero Energy Corporation (VLO)', 'News_Paragraph': 'Founded in 1980 and named for the mission San Antonio de Valero – the original name of the Alamo – Valero Energy Corporation has continued to grow and evolve to become the largest global independent petroleum refiner and the largest renewable fuels producer in North America.', 'Image_URL': 'https://www.valero.com/sites/default/files/2020-04/About_SuppliersMineralInterestInvestRecov_700X550.jpg'}


In [62]:
# EOG RESOURCES
# Define database and collection
db = client.oil_db
collection = db.eog_intro
# Upload collection
for record in eog:
    collection.insert_one(record)
# Display the MongoDB records created above
baker_news_db = db.eog_intro.find()
for record in baker_news_db:
    print(record)

{'_id': ObjectId('5fb4159c094f647d970de9b6'), 'News_Title': 'EOG Resources, Inc. (EOG)', 'News_Paragraph': 'One of the best "shale" producers is a solid bet for a potential recovery in 2021. The company s principal producing areas are located in New Mexico, North Dakota, Texas, and Wyoming in the United States; and the Republic of Trinidad and Tobago, the People s Republic of China, and Canada', 'Image_URL': 'https://s7d2.scene7.com/is/image/DNB/Industry-OilandGas-hero'}


In [20]:
# Define database and collection
db = client.oil_db
collection = db.baker_news

In [21]:
# Upload collection
for record in baker:
    collection.insert_one(record)

In [ ]:
# Display the MongoDB records created above
baker_news_db = db.baker_news.find()
for record in baker_news_db:
    print(record)

In [23]:
# Define database and collection
db = client.oil_db
collection = db.oil_news

In [24]:
# Upload collection
for record in news:
    collection.insert_one(record)

In [26]:
# Display the MongoDB records created above
oil_news_db = db.oil_news.find()
for record in oil_news_db[:50]:
    print(record)

{'_id': ObjectId('5f8247ff1b28d0739f5c7cf1'), 'Date': 'Oct 13, 2020 at 13:10 ', 'News_Title': 'Iraq Discusses Gas Investment At Its Largest Oilfield With BP', 'News_Paragraph': "Iraq discussed on Tuesday plans for investment in associated gas production at its largest oilfield, Rumaila, Iraq's Oil Minister Ihsan Abdul-Jabbar Ismail said in a statement to Shafaq News…", 'Author': ' Tsvetana Paraskova', 'Image_URL': 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-13_iitmjqvwez.jpg'}
{'_id': ObjectId('5f8247ff1b28d0739f5c7cf2'), 'Date': 'Oct 09, 2020 at 13:11 ', 'News_Title': 'Norway’s Oil Strike Could End As Soon As Friday', 'News_Paragraph': 'There is a chance that the oil workers’ strike in Norway will end as soon as on Friday, the head of the trade union whose members are on strike for…', 'Author': ' Tsvetana Paraskova', 'Image_URL': 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_zsxvky4oan.jpg'}
{'_id': ObjectId('5f8247ff1b28d0739f5c7cf3'), 'Date': 'Oct 09

In [27]:
# Define new collection
prices_collection = db.oil_prices_dec

In [28]:
# Upload collection
for record in prices:
    prices_collection.insert_one(record)

In [29]:
# Display the MongoDB records created above
oil_prices_db = db.oil_prices_dec.find()
for record in oil_prices_db[:10]:
    print(record)

{'_id': ObjectId('5fd149c52ba1680045c07f7a'), 'Oil_Symbol': 'WTI Crude', 'Oil_Price': '45.64', 'Oil_Change_Value': '+0.04', 'Oil_Change_Interest': '+0.09%', 'Oil_Price_Delay': '(10 Minutes Delay)'}
{'_id': ObjectId('5fd149c52ba1680045c07f7b'), 'Oil_Symbol': 'Brent Crude', 'Oil_Price': '48.98', 'Oil_Change_Value': '+0.14', 'Oil_Change_Interest': '+0.29%', 'Oil_Price_Delay': '(11 Minutes Delay)'}
{'_id': ObjectId('5fd149c52ba1680045c07f7c'), 'Oil_Symbol': 'Mars US', 'Oil_Price': '46.40', 'Oil_Change_Value': '-0.01', 'Oil_Change_Interest': '-0.02%', 'Oil_Price_Delay': '(21 \tHours Delay)'}
{'_id': ObjectId('5fd149c52ba1680045c07f7d'), 'Oil_Symbol': 'Opec Basket', 'Oil_Price': '47.79', 'Oil_Change_Value': '-0.56', 'Oil_Change_Interest': '-1.16%', 'Oil_Price_Delay': '(1 day Delay)'}
{'_id': ObjectId('5fd149c52ba1680045c07f7e'), 'Oil_Symbol': 'Canadian Crude Index', 'Oil_Price': '32.22', 'Oil_Change_Value': '-0.65', 'Oil_Change_Interest': '-1.98%', 'Oil_Price_Delay': '(2 \tweeks Delay)'}
{'_

In [30]:
client.close()